# Census 2017 - Population (Ingestion)

In [1]:
import sys
import configparser
config = configparser.ConfigParser()
config.read_file(open("../../settings.ini"))

sys.path.insert(0, config.get('PATHS','libs_path'))
engine_path = config.get('DATABASE','engine_path')

In [2]:
local_path = "../data_final/"
remote_path = "/"

In [3]:
# from local file postgres.py
import postgres
# from local file commons.py
from commons import inline_table_xml, inline_dimension_xml, download_file, download_zip_file, extract_zip_file

import json
import pandas as pd
from sqlalchemy import create_engine

In [4]:
df1 = download_file(remote_path, local_path, "population_census.csv")

Already downloaded. Using: ../data_final/population_census.csv


In [5]:
df1.head()

,area_id,district_id,scholarship,birth_place,residence_country,highest_course_approved,children_born_alive,age,residence_comuna,residence_5_years_ago,...,year_of_arrival_to_chile,children_currently_alive,highest_level_approved,birth_comuna,native_people,economy_activity,cant_per,comuna_datachile_id,comuna_customs_id,aboriginal_people
0,1,1,0,1,256,0,0,67,98,2,...,9998,98,4,98,2,98,1,113,1101,98
1,1,1,0,1,256,0,98,0,98,1,...,9998,98,1,98,1,98,1,113,1101,1
2,1,1,0,1,256,0,98,0,98,1,...,9998,98,1,98,2,98,6,113,1101,98
3,1,1,0,1,256,0,98,0,98,1,...,9998,98,1,98,2,98,3,113,1101,98
4,1,1,0,1,256,0,98,1,98,1,...,9998,98,1,98,1,98,1,113,1101,1


In [6]:
df1 = df1.rename(columns = {"comuna_datachile_id": "comuna_id"})

In [7]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df1, "census", "fact_population_census")

DROP TABLE IF EXISTS census.fact_population_census;
CREATE TABLE "census"."fact_population_census" (
"area_id" INTEGER,
  "district_id" INTEGER,
  "scholarship" INTEGER,
  "birth_place" INTEGER,
  "residence_country" INTEGER,
  "highest_course_approved" INTEGER,
  "children_born_alive" INTEGER,
  "age" INTEGER,
  "residence_comuna" INTEGER,
  "residence_5_years_ago" INTEGER,
  "residence_comuna_5_years_ago" INTEGER,
  "habitual_residence" INTEGER,
  "sex" INTEGER,
  "formal_education" INTEGER,
  "birth_country" INTEGER,
  "residence_country_5_years_ago" INTEGER,
  "year_of_arrival_to_chile" INTEGER,
  "children_currently_alive" INTEGER,
  "highest_level_approved" INTEGER,
  "birth_comuna" INTEGER,
  "native_people" INTEGER,
  "economy_activity" TEXT,
  "cant_per" INTEGER,
  "comuna_id" INTEGER,
  "comuna_customs_id" INTEGER,
  "aboriginal_people" INTEGER
)
COPY "census"."fact_population_census" ("area_id","district_id","scholarship","birth_place","residence_country","highest_course_app

In [8]:
engine.execute("""
CREATE INDEX fact_population_census_index 
ON census.fact_population_census (comuna_id)
""")

In [ ]:
# Read questionnaire file
questionnaire = pd.ExcelFile("https://docs.google.com/spreadsheets/d/e/2PACX-1vQ4xZxDpyDY4NursNbrsWlCqjREdmBbfC1EMlz4UGQe7M8wMA7Mqw8tZbAcBdkjgBzloyQdcnhiv10C/pub?output=xlsx")
labels = pd.read_excel(questionnaire, "Labels")

id_labels = list(df1) - []
for label in list(df1):
    q = labels[labels["label"] == label].iloc[0]["id"]
    if q[0] == "_" or q[0] == "P":
        df = pd.read_excel(questionnaire, q)
        print (inline_dimension_xml(df, label, "id", "es", label))